In [ ]:
import cv2
import torch
import easyocr
import pandas as pd
from sort.sort import Sort
import numpy as np

In [ ]:
path = 'model_weights.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path, force_reload = True)

tracker = Sort()

reader = easyocr.Reader(['en'])

In [ ]:
def detect_text(image):
    sharpening_filter = np.array([[-1, -1, -1],
                        [-1,  9, -1],
                        [-1, -1, -1]])
    image = cv2.filter2D(image, -1, sharpening_filter)
    image = cv2.fastNlMeansDenoisingColored(image, None, h = 10, hColor = 5, templateWindowSize = 16, searchWindowSize = 5) 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    area = image.shape[0]*image.shape[1]
    if (area <= 55000): 
        image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 6) 
    else: 
        image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 101, 6) 
    
    text = reader.readtext(image)
    return text

In [ ]:
# Main (Code executed in Google Colab): Writing the Processed Frames to an Output file
cap = cv2.VideoCapture("/content/drive/MyDrive/Assignment1/Test Videos/2.mp4")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./Output_2.mp4', fourcc, fps, (width, height))

ids = {}
text_ids = {}
parameters = {'RR': ('rr', 'res', 're'),
              'Sp02': ('spo', 'sp0'),
              'ECG': tuple(['ec'])}
            #   'CO2' : tuple(['co2', 'c02'])}
            #   'HR': tuple(['hr'])}
            #   'Pulse' : tuple(['Pul'])}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    dframe = np.copy(frame)

    with torch.no_grad():
        detections = model(frame).xyxy[0].cpu().numpy()

    bb_list = []
    for row in detections:
        x1, y1, x2, y2, _, _ = row
        bb_list.append([x1, y1, x2, y2])

    if bb_list:
        track_list = tracker.update(np.asarray(bb_list))
        for area in track_list:
            mx1, my1, mx2, my2, id = map(int, area)

            if id not in ids:
                ids[id] = 'NA'

            cv2.rectangle(dframe, (mx1, my1), (mx2, my2), (255, 0, 0), 2)
            cv2.putText(dframe, str(ids[id]), (mx1, my1), cv2.FONT_HERSHEY_SIMPLEX, (0.5), (0, 255, 0), 2)
            try:
                cv2.putText(dframe, f"{text_ids[id]}: {ids[id]}", (mx2-50, my1), cv2.FONT_HERSHEY_SIMPLEX, (0.5), (0, 255, 0), 2)
            except:
                pass

            mframe = frame[my1:my2, mx1:mx2]
            if mframe.size:
              scaling_factor = 2
              sframe = cv2.resize(mframe, None, fx = scaling_factor, fy = scaling_factor, interpolation = cv2.INTER_LINEAR)

              text = detect_text(sframe)
              max_area = 0
              for item in text:
                  if not item[1].isdigit():
                    # if item[1] == '?':
                    #   ids[id] = 'NA'
                    #   break
                    for key, values in parameters.items():
                      for val in values:
                          if val in item[1].lower() and id not in text_ids:
                              text_ids[id] = key
                              break
                  elif len(item[1]) < 4:
                    area = (item[0][1][0] - item[0][0][0])*(item[0][2][1] - item[0][1][1])
                    if area > max_area:
                      max_area = area
                      ids[id] = int(item[1])

    dframe = cv2.cvtColor(dframe, cv2.COLOR_RGB2BGR)
    out.write(dframe)

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# Main (Code executed in my PC): Real-Time Detection (Some frames are skipped to improve FPS performance)
cap = cv2.VideoCapture("Test Videos/2.mp4")

fcount = 0
count = 0
ids = {}
text_ids = {}

parameters = {'RR': ('rr', 'res', 're'),
              'Sp02': ('spo', 'sp0'),
              'ECG': tuple(['ec'])}
            #   'CO2' : tuple(['co2', 'c02'])}
            #   'HR': tuple(['hr'])}
            #   'Pulse' : tuple(['Pul'])}

while True:
    ret, frame = cap.read()
    if not ret: 
        break
    fcount += 1
    if fcount%9 != 0: 
        continue

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    dframe = np.copy(frame) 

    detections = pd.DataFrame(model(frame).xyxy[0])

    bb_list = []
    for _, row in detections.iterrows():
        x1, y1, x2, y2, _, _ = row
        bb_list.append([x1, y1, x2, y2])

    if bb_list:
        track_list = tracker.update(np.asarray(bb_list))
        for area in track_list:
            mx1, my1, mx2, my2, id = map(int, area)
            if id not in ids: 
                ids[id] = 'NA'
            
            cv2.rectangle(dframe, (mx1, my1), (mx2, my2), (255, 0, 0), 2)
            cv2.putText(dframe, str(ids[id]), (mx1, my1), cv2.FONT_HERSHEY_SIMPLEX, (0.5), (0, 255, 0), 2)
            try:
                cv2.putText(dframe, f"{text_ids[id]}: {ids[id]}", (mx2-50, my1), cv2.FONT_HERSHEY_SIMPLEX, (0.5), (0, 255, 0), 2)
            except:
                pass
            
            if count%9 == 0:
                mframe  = frame[my1:my2, mx1:mx2]
                scaling_factor = 2
                sframe = cv2.resize(mframe, None, fx = scaling_factor, fy = scaling_factor, interpolation = cv2.INTER_LINEAR)
                
                text = detect_text(sframe)
                max_area = 0
                for item in text:
                    if not item[1].isdigit():
                        # if item[1] == '?':
                        #     ids[id] = 'NA'
                        #     break
                        for key, values in parameters.items():
                            for val in values:
                                if val in item[1].lower() and id not in text_ids:
                                    text_ids[id] = key
                                    break
                    elif len(item[1]) < 4:
                        area = (item[0][1][0] - item[0][0][0])*(item[0][2][1] - item[0][1][1])
                        if area > max_area:
                            max_area = area
                            ids[id] = int(item[1])
                     
        count += 1
                                
    dframe = cv2.cvtColor(dframe, cv2.COLOR_RGB2BGR)
    cv2.imshow("Frame", dframe)
    
    if cv2.waitKey(1) & 0xFF == 27: 
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)